<a href="https://colab.research.google.com/github/Nacho2904/orga_de_datos/blob/main/tp3_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP3 Parte II: Baseline

Del análisis exploratorio de la parte I hemos aprendido algunas cosas:
- Hay algunas columnas que no nos aportan información. En particular *did* tiene utilidad por no estar en la mayoría de columnas, y *s-label* no sabemos cómo interpretarlo. *Language* tampoco aporta demasiado debido a que falta en muchas canciones 

- Algunas columnas requieren preprocessing. Las tres columnas de texto que tenemos, *track-name*, *lyrics* y *artist*, no pueden ser usadas directamente. *mode* y *key*, por otro lado, son features categóricas. *a_genres* también es una variable categórica que contiene varias clases. Luego hay que preprocesar las features de texto para crear nuevos features útiles, y preprocesar las features categóricas para poder utilizarlas en la regresión logística.

- Noo tenemos suficientes observaciones para algunos de los posibles valores del target.

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from google.colab import drive 

drive.mount('/content/gdrive')
path_a_training_set = 'gdrive/MyDrive/TP3 dataset music/train.parquet'

df_music_train = pd.read_parquet(path_a_training_set)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [62]:
df_music_train_filtered = df_music_train.drop(labels=["s-label", "did", "language"], axis=1)
df_music_train_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31383 entries, 0 to 34336
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_name        31383 non-null  object 
 1   lyric             31380 non-null  object 
 2   genre             31383 non-null  object 
 3   popularity        31383 non-null  int64  
 4   artist            31383 non-null  object 
 5   a_genres          31383 non-null  object 
 6   a_songs           31383 non-null  float64
 7   a_popularity      31383 non-null  float64
 8   acousticness      31383 non-null  float64
 9   danceability      31383 non-null  float64
 10  duration_ms       31383 non-null  int64  
 11  energy            31383 non-null  float64
 12  instrumentalness  31383 non-null  float64
 13  key               31383 non-null  object 
 14  liveness          31383 non-null  float64
 15  loudness          31383 non-null  float64
 16  mode              31383 non-null  object

En primer lugar notamos que las variables categóricas *key* y *time-signature* ambas son ordinales. La primera representa el tono dominante en la canción., y tomaremos el orden dado en [este blog](https://viva.pressbooks.pub/openmusictheory/chapter/pitch-and-pitch-class/). El *time-signature* es una medida de la cantidad de pulsos por unidad, y también está ordenado naturalmente. *Mode* es una variable binaria así que la encodeamos como 0 y 1.

In [40]:
ordinalEncoder = preprocessing.OrdinalEncoder(categories = [["Minor", "Major"],['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'],
                                                            ['1/4', '3/4', '4/4', '5/4']])
ordinalEncoder.fit(df_music_train_filtered[["mode","key", "time_signature"]])

OrdinalEncoder(categories=[['Minor', 'Major'],
                           ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#',
                            'A', 'A#', 'B'],
                           ['1/4', '3/4', '4/4', '5/4']])

Para las letras de los se me ocurre aplicar algo de NLP para realizar sentiment analysis.

In [69]:
import nltk
nltk.download(["names", "stopwords", "averaged_perceptron_tagger", "vader_lexicon", "punkt"])

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [90]:
stopwords = set(nltk.corpus.stopwords.words("english")).union(set(nltk.corpus.stopwords.words("spanish"))).union(set(nltk.corpus.stopwords.words("french")))
vocab_from_lyrics = df_music_train_filtered["lyric"].map(lambda lyric: [w.lower() for w in lyric.split(' ') if w.isalpha() and w.lower() not in stopwords]
                                                         if lyric else [])
vocab_from_lyrics

0        [remember, walls, baby, tumbling, even, put, e...
1        [remember, walls, baby, tumbling, even, put, e...
2        [remember, walls, baby, tumbling, even, put, e...
3        [driver, roll, partition, roll, partition, nee...
4        [driver, roll, partition, roll, partition, nee...
                               ...                        
34332    [side, side, always, verita, side, always, ver...
34333    [amor, mio, cuentare, nadia, volvere, volvere,...
34334                                           [bolerias]
34335    [quiero, ser, si, vez, quiero, ser, si, siempr...
34336    [something, happening, exactly, clear, man, gu...
Name: lyric, Length: 31383, dtype: object

In [64]:
from sklearn.compose import ColumnTransformer

text_features = ["track_name", "lyric", "artist"]

numerical_features = ["a_songs", "a_popularity", "popularity", "acousticness", "danceability", "duration_ms",
                   "energy", "instrumentalness", "liveness", "loudness", "speechiness", "tempo", "valence"]

ordinal_features = ["mode", "key", "time_signature"]

identity_transformer = preprocessing.FunctionTransformer(None)

full_processor = ColumnTransformer(transformers=[
    ('text', identity_transformer, text_features),
    ('numerical', identity_transformer, numerical_features),
    ('ordinal', ordinalEncoder, ordinal_features)
])

pd.DataFrame(full_processor.fit_transform(df_music_train_filtered))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Halo,Remember those walls I built\nWell baby they'r...,Beyoncé,276.0,205.5,79.0,0.272,0.508,261640.0,0.72,0.0,0.0563,-5.908,0.0628,79.983,0.472,0.0,11.0,2.0
1,Halo,Remember those walls I built\nWell baby they'r...,Beyoncé,276.0,205.5,80.0,0.272,0.508,261640.0,0.72,0.0,0.0563,-5.908,0.0628,79.983,0.472,0.0,11.0,2.0
2,Halo,Remember those walls I built\nWell baby they'r...,Beyoncé,276.0,205.5,80.0,0.272,0.508,261640.0,0.72,0.0,0.0563,-5.908,0.0628,79.983,0.472,0.0,11.0,2.0
3,Partition,Driver roll up the partition please\nDriver ro...,Beyoncé,276.0,205.5,71.0,0.0296,0.412,319467.0,0.441,0.0726,0.306,-11.523,0.291,185.571,0.174,0.0,11.0,2.0
4,Partition,Driver roll up the partition please\nDriver ro...,Beyoncé,276.0,205.5,71.0,0.0296,0.412,319467.0,0.441,0.0726,0.306,-11.523,0.291,185.571,0.174,0.0,11.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31378,A Tu Vera,To your side To your side always the verita yo...,Gipsy Kings,89.0,1.0,19.0,0.612,0.616,190733.0,0.822,0.000003,0.165,-8.063,0.0561,119.962,0.748,0.0,4.0,2.0
31379,No Volvere,Amor mio\r\nAmor mio por favor\r\nTu no te vas...,Gipsy Kings,89.0,1.0,34.0,0.889,0.457,230200.0,0.369,0.0111,0.231,-12.515,0.0353,80.214,0.868,0.0,4.0,2.0
31380,Bolerias,Bolerias is instrumental\n\n\n\n,Gipsy Kings,89.0,1.0,18.0,0.763,0.717,275640.0,0.566,0.812,0.144,-14.2,0.0405,117.321,0.793,0.0,6.0,1.0
31381,Como Ayer,Yo quiero ser si otra vez llegaras\r\nYo quier...,Gipsy Kings,89.0,1.0,28.0,0.804,0.633,204373.0,0.553,0.000866,0.139,-8.851,0.0376,87.608,0.738,1.0,0.0,2.0
